---
# Notebook para automatizar fiscalização - Específico para Empresas do Simples Nacional
---

Esse notebook tem por propósito automatizar a fiscalização por monitoramento.

## 1. Configuração Inicial

### 1.1. Configurações Iniciais

#### 1.1.1. Importação de bibliotecas

In [1]:
import pandas as pd 
from datetime import datetime as dt 
from dateutil.relativedelta import relativedelta
from connection import DadosEscriturais, DadosOperacoes, DadosCadastrais, DadosParametrizados, DadosDebitos
from external_connection import DadosCadastraisRFB, DadosCadastraisJUCERR
from EFD.EFDLocal import EFDLocal
import os 
import warnings
from report_generator import RelatorioAuditoria


relatorio = {}
anexos = {}
indice_anexo = 1


#### 1.1.2. Configurações básicas

In [2]:
data_inicio=dt(2022, 1, 1)
data_fim=dt(2023, 12, 31)
cnpj = '29563548000121'
cgf = '24.033751-5'

valor_minimo_significancia = 10000

endereco_pasta_audit = r'C:\Users\jimmy.gomes\Documents\Planos de Trabalho\PT02-2024\Fiscalização\SEI nº 22101014505202431\Dados'

doc = RelatorioAuditoria(cnpj='00000000000100', 
                         cgf='24-000.212', 
                         ordem_servico = '20123120309', 
                         proc_sei = '912839128301293812093', 
                         data_inicio = dt(2022, 1, 1), 
                         data_fim = dt(2023, 12, 31), 
                         nome_arquivo = f'relatorio_preliminar_auditoria_{cnpj}')


##### 1.1.2.1. Tratamentos adicionais

In [3]:
num_months = (data_fim.year - data_inicio.year) * 12 + data_fim.month - data_inicio.month + 1   # Calcular quantidade de meses de fiscalização
meses_audit = [(data_inicio + relativedelta(months=x)).strftime('%m/%Y') for x in range(num_months)] # Transformar quantidade de meses de fiscalização em formato MM/YYYY

warnings.filterwarnings('ignore')   # Ignorar alguns avisos de deprecação de funções do pandas

### 1.2. Coleta de PGDAS e GIM

In [4]:
# Coleta de GIM
gim = DadosEscriturais.GIM(cgf=cgf, data_inicio=data_inicio, data_fim=data_fim)
obrigatoriedade = DadosEscriturais.BuscarObrigatoriedade(cgf=cgf, data_inicio=data_inicio, data_fim=data_fim)

# Salvar GIM importada (para documentação de auditoria)
gim.save(path=endereco_pasta_audit+r'\escrituracao_GIM.xlsx')


# Coleta de PGDAS 
# pgdas = DadosEscriturais.PGDAS(cnpj=cnpj, data_inicio=data_inicio, data_fim=data_fim)
# pgdas.save(path=endereco_pasta_audit+r'\escrituracao_PGDAS.xlsx', sheet_name='PGDAS')


Recuperado 0 registros!


In [5]:
# Coleta de EFD
# dados_efd = EFDLocal(cnpj=cnpj, mes_ref=meses_audit, local=endereco_efd)

### 1.3. Coleta de documentos adicionais

In [6]:
parametrizacoes = DadosParametrizados

### 1.4. Coleta da Inscrição no CNPJ

In [7]:
dados_cnpj = DadosCadastraisRFB(cnpj=cnpj)
dados_jucerr = DadosCadastraisJUCERR(cnpj=cnpj)


### 1.5. Coleta da inscrição no CGF

In [8]:
cadastro = DadosCadastrais.CadastroAtualEstadual(cnpj)
cadastro_historico = DadosCadastrais.HistoricoCadastroEstadual(cnpj)

cadastro.save(path=endereco_pasta_audit+r'\cadastro_atual_estadual.xlsx')
cadastro_historico.save(path=endereco_pasta_audit+r'\historico_cadastro_estadual.xlsx')

Recuperado 1 registros!
Recuperado 3 registros!


### 1.6. Coleta de Dsot e Pagamentos Antecipados

In [9]:
dsot = DadosDebitos.DSOT(cgf, data_inicio, data_fim)
pag_antecipado = DadosDebitos.DebitosAntecipacaoParcialPagos(cgf, data_inicio, data_fim)

dsot.save(path=endereco_pasta_audit+r'\dsot_periodo.xlsx', sheet_name='DSOT')
# pag_antecipado.save(path=endereco_pasta_audit+r'\pag_antecipado_periodo.xlsx', sheet_name='PagamentoAntecipado')

Recuperado 0 registros!
Recuperado 24 registros!


### 1.7. Coleta de Documentos Fiscais Não Escriturais

In [10]:
NFeEntrada = DadosOperacoes.NotasFiscaisEntrada(cnpj=cnpj, data_inicio=data_inicio, data_fim=data_fim)
NFeSaida = DadosOperacoes.NotasFiscaisSaida(cnpj=cnpj, data_inicio=data_inicio, data_fim=data_fim)
NFCe = DadosOperacoes.NotasFiscaisConsumidor(cnpj=cnpj, data_inicio=data_inicio, data_fim=data_fim)

#### 1.7.1. Salvando arquivos para consulta posterior

In [11]:
# Salvar Documentação de Auditoria
# NFeEntrada.save(path=endereco_pasta_audit+r'\notas_fiscais_entrada.xlsx', sheet_name='NFe_Entrada')
# NFeSaida.save(path=endereco_pasta_audit+r'\notas_fiscais_saida.xlsx', sheet_name='NFe_Saida')
# NFCe.save(path=endereco_pasta_audit+r'\notas_fiscais_ao_consumidor.xlsx', sheet_name='NFCE')


## 2. Auditoria de Obrigações Acessórias

In [12]:
relatos = {}
explicacoes_adicionais = []

### 2.1. Verificação da inscrição no CNPJ

- Verificar se está inscrito;
- Verificar o período de inscrição;


In [13]:
temp = dados_cnpj.BuscarCadastro()

relatos = {}


if temp['SituacaoCadastral'].values in ('Ativa', 'Ativo'):
    relatos['Verificação da Inscrição no CNPJ'] = [f'Contribuinte está inscrito no CNPJ desde {pd.to_datetime(temp["DataCadastro"].values[0]).strftime("%d/%m/%Y")}.']
    del temp

else:
    relatos['Verificação da Inscrição no CNPJ'] = [f'Contribuinte está com inscrição desativada conforme Anexo {indice_anexo}.']
    anexos[f'Anexo {indice_anexo} - Cadastro Desativado'] = [temp]
    indice_anexo += 1


explicacao_adicional = f'Foi acessado o banco de dados CNPJ da Receita Federal no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de confirmar a situação da auditada, especificamente se a mesma está com inscrição ativa no órgão federal. '
explicacoes_adicionais.append(explicacao_adicional)

### 2.2. Verificação da inscrição no CGF 

- Verificar status da inscrição estadual no período de abrangência
- Verificar regime de apuração
- Verificar se o sócio-administrador está batendo com banco estadual

In [14]:
temp = cadastro_historico.retrieve()

relatos = {}

if temp['SituacaoCadastral'].values[-1] in ('A', 'N'):
    relatos['Verificação da Inscrição no Cadastro Geral da Fazenda'] = [f'Contribuinte está ativo no CGF.']
    # del temp 

else:
    relatos['Verificação da Inscrição no Cadastro Geral da Fazenda'] =  [f'Contribuinte está com inscrição pendente, conforme Anexo II. Situação Cadastral atual: {temp["SituacaoCadastral"].values[-1]}.']
    anexos[f'Anexo {indice_anexo} - Histórico de Cadastro Geral da Fazenda'] = [temp]
    indice_anexo += 1


explicacao_adicional = f'Foi acessado o banco de dados cadastrais da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de confirmar a situação cadastral da auditada, especificamente se a mesma está com inscrição ativa no órgão estadual. '

explicacoes_adicionais.append(explicacao_adicional)

Recuperado 3 registros!


In [15]:
# raise NotImplementedError('Ainda não está pronto! Falta importar Socios do CGF!')
socios_jucerr = dados_jucerr.BuscarSocios()
# socios_cgf = DadosCadastrais.BuscarSocios(cnpj)

### 2.3. Verificação de Emissão de Documentos Fiscais


In [16]:
relatos['Verificação de Emissão de Documentos Fiscais'] = []

if NFeEntrada.quantidade()['QUANTIDADE'].values == 0:
    msg = 'O contribuinte NÃO está recebendo notas fiscais (de entrada) no período de abrangência.'
    relatos['Verificação de Emissão de Documentos Fiscais'].append(msg)
else:
    msg = 'O contribuinte está recebendo notas fiscais (de entrada) no período de abrangência.'
    relatos['Verificação de Emissão de Documentos Fiscais'].append(msg)
    
print(msg)

if NFeSaida.quantidade()['QUANTIDADE'].values == 0 and NFCe.quantidade()['QUANTIDADE'].values == 0:
    msg = 'O contribuinte NÃO está emitindo nota fiscal de saída (inclusive NFCe)!'
    relatos['Verificação de Emissão de Documentos Fiscais'].append(msg)

else:
    msg = 'O contribuinte está emitindo Nota Fiscal Eletrônica de saída / Nota Fiscal ao Consumiddor Eletrônica!'
    relatos['Verificação de Emissão de Documentos Fiscais'].append(msg)

print(msg)

explicacao_adicional = f'Foi acessado o banco de dados da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de confirmar se a auditada está realizando emissões de documentos fiscais (de saída) e se está recebendo notas fiscais (de entrada). Esta validação, contudo, se restringe às operações com mercadorias, não adentrando no espectro de emissão de documentos fiscais relativos à energia elétrica, serviços de transporte e serviços de comunicação.'

explicacoes_adicionais.append(explicacao_adicional)

O contribuinte está recebendo notas fiscais (de entrada) no período de abrangência.
O contribuinte está emitindo Nota Fiscal Eletrônica de saída / Nota Fiscal ao Consumiddor Eletrônica!


### 2.4. Verificação da declaração da Guia de Informação Mensal do ICMS

In [17]:
# raise NotImplementedError('Excluir meses em que contribuinte estava pelo Simples Nacional')
data_atual = data_inicio
lista_periodos_sem_gim = []
obrigatoriedade_gim = [data.strftime('%m/%Y') for data in obrigatoriedade[obrigatoriedade['OB_TIPO']=='GIM']['OB_DATA']]


gim_importada = gim.retrieve()
while data_atual <= data_fim:
    if data_atual.strftime('%m/%Y') not in gim_importada["PERÍODO"].values and data_atual.strftime('%m/%Y') in obrigatoriedade_gim:
        lista_periodos_sem_gim.append(data_atual.strftime('%m/%Y'))
    data_atual += relativedelta(months=1)

if len(lista_periodos_sem_gim) != 0:
    msg = 'Há meses com omissão de declaração de Guia de Informações Mensais no período de abrangência.'
    relatos['Verificação de Declaração de GIM'] = [msg]
    
elif len(obrigatoriedade_gim) == 0:
    msg = 'O contribuinte não esteve obrigado a emitir qualquer Guia de Informações Mensais no período de abrangência.'
    relatos['Verificação de Declaração de GIM'] = [msg]
    
else:
    msg = f'O contribuinte transmitiu todas as {len(obrigatoriedade_gim)} Guia de Informações Mensais obrigatórias no período de abrangência.'
    relatos['Verificação de Declaração de GIM'] = [msg]
    del gim_importada

print(msg)

explicacao_adicional = f'Foi acessado o banco de dados da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de identificar se a auditada está declarando Guias de Informações Mensais de ICMS nos períodos em que consta como obrigatórios. Não são validados, contudo, os valores declarados, mas tão somente a emissão de per si.'

explicacoes_adicionais.append(explicacao_adicional)

O contribuinte não esteve obrigado a emitir qualquer Guia de Informações Mensais no período de abrangência.


### 2.5. Verificação da Declaração de PGDAS no período 

In [18]:
# raise NotImplementedError 

### 2.6. Relatar resultado de Auditoria de Obrigações Acessórias

In [19]:
nome_metodo = 'Auditoria de Obrigações Acessórias e Cadastrais'
texto = f"No dia {dt.today().strftime('%d/%m/%Y')}, foi realizada auditoria de cumprimento das obrigações acessórias e cadastrais em geral, por meio das seguintes verificações: \n \n"
for relato in relatos.keys():
    texto += f'- {relato} \n '

existe_explicacoes = False 
texto_adicional = ''
for relato in explicacoes_adicionais:
    texto_adicional += f'{relato} \n'
    existe_explicacoes = True 
if not existe_explicacoes:
    texto_adicional = None



resultado = 'Por meio dos procedimentos acima, chegou-se às seguintes conclusões: \n \n'
for relato in relatos.values():
    for resultado_parcial in relato:
        resultado += f' - {resultado_parcial} \n '

del relatos


# Adicionar ao relatório 
doc.inserir_procedimento(nome_metodo=nome_metodo, texto=texto, texto_adicional=texto_adicional, resultado=resultado)

## 3. Auditoria dos Dados Cadastrais

In [20]:
# raise NotImplementedError

## 4. Auditoria da Situação Fiscal

### 4.1. Verificação da existência de débitos constantes do DSOT


---
<center> Fundamentos Legais </center>

---
 

Art. 81. A exclusão do Simples Nacional, mediante comunicação da ME ou da EPP à RFB, em aplicativo disponibilizado no Portal do Simples Nacional, dar-se-á:

[...]

II - obrigatoriamente, quando:

[...]

d) possuir débito com o Instituto Nacional do Seguro Social (INSS), ou com as Fazendas Públicas Federal, Estadual ou Municipal, cuja exigibilidade não esteja suspensa, hipótese em que a exclusão: (Lei Complementar nº 123, de 2006, art. 17, inciso V; art. 30, inciso II)
1. deverá ser comunicada até o último dia útil do mês subsequente ao da situação de vedação; e (Lei Complementar nº 123, de 2006, art. 30, § 1º, inciso II)
2. produzirá efeitos a partir do ano-calendário subsequente ao da comunicação; ou(Lei Complementar nº 123, de 2006, art. 31, inciso IV)


[...]

---

Art. 84. A exclusão de ofício da ME ou da EPP do Simples Nacional produzirá efeitos:

[...]

V - a partir do primeiro dia do mês seguinte ao da ocorrência, na hipótese de ausência ou irregularidade no cadastro fiscal federal, municipal ou, quando exigível, estadual; e (Lei Complementar nº 123, de 2006, art. 17, inciso XVI; art. 31, inciso II)

[...]

VI - a partir do ano-calendário subsequente ao da ciência do termo de exclusão, se a empresa estiver em débito com o Instituto Nacional do Seguro Social (INSS), ou com as Fazendas Públicas Federal, Estadual ou Municipal, cuja exigibilidade não esteja suspensa. (Lei Complementar nº 123, de 2006, art. 17, inciso V; art. 31, inciso IV)

[...]

§ 1º Na hipótese prevista nos incisos V e VI do caput, a comprovação da regularização do débito ou do cadastro fiscal, no prazo de até 30 (trinta) dias, contado da ciência da exclusão de ofício, possibilitará a permanência da ME ou da EPP como optante pelo Simples Nacional. (Lei Complementar nº 123, de 2006, art. 31, § 2º)

In [21]:
dsot_existente = dsot.retrieve()
dsot_ativo = dsot_existente[dsot_existente['Suspensao']=='N']
relatos = {}
explicacoes_adicionais = []

if dsot_existente.empty:
    msg = 'Não há débitos registrados nos sistemas da SEFAZ-RR.'
    del dsot_ativo
    del dsot_existente

elif dsot_ativo.empty: 
    msg = f'Há apenas débitos suspensos registrados nos sistemas da SEFAZ-RR, conforme Anexo {indice_anexo}'
    anexos[f'Anexo {indice_anexo} - Tabelas de Débitos Fiscais em Aberto'] = [dsot_existente]
    indice_anexo += 1
    del dsot_ativo

else: 
    valor_total_debito = sum(dsot_ativo['ValorPrincipal'] + dsot_ativo['ValorMulta'] + dsot_ativo['ValorJuros'] + dsot_ativo['ValorCorrecao'])
    
    msg = f'Há débitos registrados nos sistemas da SEFAZ-RR no montante de {valor_total_debito}, conforme Anexo {indice_anexo}'
    anexos[f'Anexo {indice_anexo} - Tabelas de Débitos Fiscais em Aberto'] = [dsot_ativo] 
    indice_anexo += 1
    del dsot_existente


print(msg)
relatos['Verificação de Débitos Fiscais em Aberto, conforme banco de dados estadual'] = [msg]


explicacao_adicional = f'Foi acessado o banco de dados da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de identificar se a auditada possui pendências fiscais (débitos em aberto) nos períodos em auditoria. Não são validados, contudo, os valores em débitos, mas tão somente a existência dos mesmos.'

explicacoes_adicionais.append(explicacao_adicional)


Não há débitos registrados nos sistemas da SEFAZ-RR.


### 4.2. Relatar resultados 

In [22]:
nome_metodo = 'Auditoria de Situação Fiscal'
texto = f"No dia {dt.today().strftime('%d/%m/%Y')}, foi realizada auditoria de situação fiscal, por meio das seguintes verificações: \n "

for relato in relatos.keys():
    texto += f'{relato} \n '

existe_explicacoes = False 
texto_adicional = ''
for relato in explicacoes_adicionais:
    texto_adicional += f'{relato} \n'
    existe_explicacoes = True 
if not existe_explicacoes:
    texto_adicional = None


resultado = ' '
for relato in relatos.values():
    for resultado_parcial in relato:
        resultado += f'- {resultado_parcial} \n'

del relatos

# Adicionar ao relatório 
doc.inserir_procedimento(nome_metodo=nome_metodo, texto=texto, texto_adicional = texto_adicional, resultado=resultado)

## 5. Auditoria das Operações e Prestações de Entrada

Objetivos:
- Certificar desembaraço de notas fiscais 
- Certificar aquisições em relação ao faturamento


---
<center> Fundamentos Legais </center>

---


Art. 84. A exclusão de ofício da ME ou da EPP do Simples Nacional produzirá efeitos:

[...]

IV - a partir do próprio mês em que incorridas, hipótese em que a empresa ficará impedida de fazer nova opção pelo Simples Nacional nos 3 (três) anos-calendário subsequentes, nas seguintes hipóteses: (Lei Complementar nº 123, de 2006, art. 29, incisos II a XII e § 1º)

[...]

h) se for constatado que durante o ano-calendário o valor das despesas pagas supera em 20% (vinte por cento) o valor de ingressos de recursos no mesmo período, excluído o ano de início de atividade;

i) se for constatado que durante o ano-calendário o valor das aquisições de mercadorias para comercialização ou industrialização, ressalvadas hipóteses justificadas de aumento de estoque, foi superior a 80% (oitenta por cento) dos ingressos de recursos no mesmo período, excluído o ano de início de atividade;

[...]

§ 6º Considera-se prática reiterada, para fins do disposto nas alíneas “d”, “j” e “k” do inciso IV do caput: (Lei Complementar nº 123, de 2006, art. 29, § 9º)

I - a ocorrência, em 2 (dois) ou mais períodos de apuração, consecutivos ou alternados, de idênticas infrações, inclusive de natureza acessória, verificada em relação aos últimos 5 (cinco) anos-calendário, formalizadas por intermédio de auto de infração ou notificação de lançamento, em um ou mais procedimentos fiscais; ou

II - a segunda ocorrência de idênticas infrações, caso seja constatada a utilização de artifício, ardil ou qualquer outro meio fraudulento que induza ou mantenha a fiscalização em erro, com o fim de suprimir ou reduzir o pagamento de tributo.

In [23]:
# Trazer dados para memória RAM 
nfe_entrada = NFeEntrada.retrieve()
eventos_desemb = DadosOperacoes.EventosDesembaraco(tuple(nfe_entrada[nfe_entrada['DESTINO']=='INTERESTADUAL']['CHAVE_ACESSO'].values)).retrieve()
eventos = DadosOperacoes.EventosNotas(tuple(nfe_entrada['CHAVE_ACESSO'].values)).retrieve()

Recuperado 11811 registros!
Recuperado 491 registros!
Recuperado 6101 registros!


In [24]:
# Iniciar as variáveis de relatório
relatos = {}
explicacoes_adicionais = []

### 5.1. Verificação de notas fiscais não desembaraçadas 

Cria-se uma tabela com as notas fiscais não desembaraçadas

Regras:
- Data de Desembaraço: Data de emissão + 90
- Remove notas com passagem no posto e com código de evento de cancelamento.

In [25]:

# Primeira parte - Obtendo documentos interestaduais
nfe_n_desemb = nfe_entrada[(nfe_entrada['DESTINO']=='INTERESTADUAL')&(nfe_entrada['FINALIDADE'].isin(['NORMAL', 'DEVOLUÇÃO', 'N/A']))]

# Segunda parte - Obtendo eventos de desembaraço para filtrar por não desembaraço
nfe_n_desemb = nfe_n_desemb[~nfe_n_desemb['CHAVE_ACESSO'].isin(eventos_desemb['CHAVE_ACESSO'])]


# Terceira Parte Encontrando data de desembaraço devida
nfe_n_desemb['DataDesembaraçoPrevista'] = [linha + relativedelta(days=90) for linha in nfe_n_desemb['DT_EMISSAO']]


# Quarta Parte - Filtrando pelos documentos com eventos
eventos_canc = eventos[eventos['CODIGO_EVENTO'].isin(['210220', '210240'])]
nfe_n_desemb = nfe_n_desemb[~(nfe_n_desemb['CHAVE_ACESSO'].isin(eventos_canc['CHAVE_ACESSO']))]

# Sexta parte - Indicar existência de notas não desembaraçadas
if not nfe_n_desemb.empty: 
    msg = f'Foram encontradas {len(nfe_n_desemb)} Notas Fiscais válidas e não desembaraçadas no período, conforme anexo {indice_anexo}'
    anexos[f'Anexo {indice_anexo} - Notas Fiscais Interestaduais Não Desembaraçadas'] = [nfe_n_desemb]
    indice_anexo += 1 

else:
    msg = f'Não foram encontradas Notas Fiscais não desembaraçadas no período.'
    
    del nfe_n_desemb

relatos['Verificação de Notas Fiscais Não Desembaraçadas'] = [msg]
print(msg)
# Sexta parte - Salvar conjunto de notas não desembaraçadas
del eventos_canc
del eventos_desemb

explicacao_adicional = f'Foi acessado o banco de dados da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de identificar se a auditada possui Notas Fiscais Eletrônicas interestaduais não desembaraçadas. Nesse contexto, são considerados os eventos de Desconhecimento da Operação, Não Realização da Operação e Cancelamentos. Não são validados, contudo, os valores lançados no documento fiscal.'

explicacoes_adicionais.append(explicacao_adicional)

Foram encontradas 1346 Notas Fiscais válidas e não desembaraçadas no período, conforme anexo 1


### 5.2. Relatar 

In [26]:
nome_metodo = 'Auditoria de Operações e Prestações de Entrada'
texto = f"No dia {dt.today().strftime('%d/%m/%Y')}, foi realizada auditoria de Operações e Prestações de Entrada, por meio das seguintes verificações: \n "

for relato in relatos.keys():
    texto += f'{relato} \n '

existe_explicacoes = False 
texto_adicional = ''
for relato in explicacoes_adicionais:
    texto_adicional += f'{relato} \n'
    existe_explicacoes = True 
if not existe_explicacoes:
    texto_adicional = None


resultado = ' '
for relato in relatos.values():
    for resultado_parcial in relato:
        resultado += f'- {resultado_parcial} \n'

del relatos

# Adicionar ao relatório 
doc.inserir_procedimento(nome_metodo=nome_metodo, texto=texto, resultado=resultado)

## 6. Auditoria de Operações e Prestações de Saída

Objetivo: 
- Certificar ausência de desempenho negativo (por mês)
- Certificar ausência de desempenho negativo (por produto)
- Certificar emissão de documentos fiscais 

---
<center> Fundamentos Legais </center>

---
> RICMS-RR

<b>Art. 858.</b> Para apuração das operações ou prestações realizadas pelo sujeito passivo, o fisco
poderá utilizar quaisquer procedimentos tecnicamente idôneo, tais como:

I – análise da escrita comercial e fiscal e de documentos fiscais e subsidiários;

[...]

<b>Art. 859.</b> Presumir-se-á operação ou prestação tributável não registrada, quando se constatar:

[...]

II – diferença apurada pelo cotejo entre as saídas registradas e o valor das saídas a preço de
custo acrescido do lucro apurado mediante a aplicação de percentual fixado em portaria do Secretário de
Estado da Fazenda; * Fixado pela SEFAZ/PORTARIA/GAB nº 137/2015, em 30% (trinta por cento)

[...]


VI – diferença apurada mediante controle quantitativo de mercadorias, assim entendido o
confronto entre a quantidade de unidades estocadas e as quantidades de entradas e de saídas;

[...]

<b>Art. 860.</b> Constatada, por indicação na escrituração do contribuinte ou outro qualquer elemento
de prova, a saída de mercadoria ou a prestação de serviço sem emissão de documento fiscal, a autoridade
fiscal deve arbitrar o valor da operação ou da prestação.

Parágrafo único. Para efeito de arbitramento da base de cálculo do imposto e de multa, sem
prejuízo do disposto no artigo 37, será tomada como critério a média ponderada dos preços unitários das
saídas ou entradas verificadas no período.


---

> Resolução CGSN 140/2018

<b>Art. 84.</b> A exclusão de ofício da ME ou da EPP do Simples Nacional produzirá efeitos:

[...]

IV - a partir do próprio mês em que incorridas, hipótese em que a empresa ficará impedida de fazer nova opção pelo Simples Nacional nos 3 (três) anos-calendário subsequentes, nas seguintes hipóteses: (Lei Complementar nº 123, de 2006, art. 29, incisos II a XII e § 1º)


[...]

j) se for constatado que a empresa, de forma reiterada, não emite documento fiscal de venda ou prestação de serviço, observado o disposto nos arts. 59 a 61 e ressalvadas as prerrogativas do MEI nos termos da alínea "a" do inciso II do art. 106; e

In [27]:
# Trazer dados para memória RAM 
nfe_entrada = NFeEntrada.retrieve()
nfe_saida = NFeSaida.retrieve()
nfce = NFCe.retrieve()

eventos_nfe_entrada = DadosOperacoes.EventosNotas(tuple(nfe_entrada['CHAVE_ACESSO'].values)).retrieve()
eventos_nfe_saida = DadosOperacoes.EventosNotas(tuple(nfe_saida['CHAVE_ACESSO'].values)).retrieve()
eventos_nfce = DadosOperacoes.EventosNotasConsumidor(tuple(nfce['CHAVE_ACESSO'].values)).retrieve()

Recuperado 11811 registros!
Recuperado 7314 registros!
Recuperado 22114 registros!
Recuperado 6101 registros!
Recuperado 81 registros!
Recuperado 0 registros!


In [28]:
relatos = {}
explicacoes_adicionais = []


In [29]:
def excluir_notas_canceladas(doc, eventos):
    """ Eventos para exclusão de notas com eventos de cancelamento / desconhecimento """
    # Obter últimos eventos
    ultimo_evento = eventos.groupby(['CHAVE_ACESSO'], as_index=False)[['SEQUENCIA_EVENTO']].max()
    eventos_validos = eventos[['CHAVE_ACESSO', 'SEQUENCIA_EVENTO','CODIGO_EVENTO']].merge(ultimo_evento[['CHAVE_ACESSO', 'SEQUENCIA_EVENTO']], on = ['CHAVE_ACESSO', 'SEQUENCIA_EVENTO'], how='inner')

    # Filtrar eventos de cancelamento
    eventos_canc = eventos_validos[eventos_validos['CODIGO_EVENTO'].isin(['210220', '210240'])]

    # Filtrar docs que não possuem eventos
    doc_filtrado = doc.merge(eventos_canc, on = 'CHAVE_ACESSO', how='left', indicator=True)
    doc_filtrado = doc_filtrado[doc_filtrado['_merge']=='left_only']
    doc_filtrado.drop(columns=['_merge', 'SEQUENCIA_EVENTO', 'CODIGO_EVENTO'], inplace=True)

    return doc_filtrado


def inverter_cnpj_em_notas_entradas(doc):
    # Inverter CNPJ_REM e CNPJ_DEST se TIPO_OPER = 'ENTRADA'
    if 'TIPO_OPER' in doc.columns and 'CNPJ_ALTERADO' not in doc.columns:

        # Invertendo CNPJ        
        doc['CNPJ_REM_MOD'] = [linha['CNPJ_DEST'] if linha['TIPO_OPER']=='ENTRADA' else linha['CNPJ_REM'] for _, linha in doc.iterrows()]
        doc['CNPJ_DEST_MOD'] = [linha['CNPJ_REM'] if linha['TIPO_OPER']=='ENTRADA' else linha['CNPJ_DEST'] for _, linha in doc.iterrows()]

        doc.drop(columns=['CNPJ_DEST', 'CNPJ_REM'], inplace=True)
        doc.rename(columns={'CNPJ_REM_MOD':'CNPJ_REM', 'CNPJ_DEST_MOD': 'CNPJ_DEST'}, inplace=True)
        doc['CNPJ_ALTERADO'] = ['SIM' if linha['TIPO_OPER']=='ENTRADA' else 'NÃO' for _, linha in doc.iterrows()]

    elif 'CNPJ_ALTERADO' in doc.columns:
        print('Não invertido em virtude de já haver operação anterior de inversão.')

    else:
        print('Não invertido por outro erro qualquer.')
    return doc


In [30]:
nfe_entrada = excluir_notas_canceladas(nfe_entrada, eventos_nfe_entrada)
nfe_entrada = inverter_cnpj_em_notas_entradas(nfe_entrada)

nfe_saida = excluir_notas_canceladas(nfe_saida, eventos_nfe_saida)
nfe_saida = inverter_cnpj_em_notas_entradas(nfe_saida)

# Adequar nfe_saida para ter apenas saídas e nfe_entrada para ter apenas entrada (migrar dados de cnpj invertidos entre as variáveis)
nfe = pd.concat([nfe_entrada, nfe_saida])
nfe_entrada = nfe[nfe['CNPJ_DEST']==cnpj]
nfe_saida = nfe[nfe['CNPJ_REM']==cnpj]

del nfe 

### 6.1. Verificação de Desempenho Negativo (Por Mês)


Art. 75. Os contribuintes do ICMS localizados neste Estado, que adquirirem mercadorias
oriundas de outras unidades da Federação, ficam sujeitos ao recolhimento antecipado do imposto relativo à
diferença entre a alíquota interna e a interestadual, pelas operações que venham realizar no território deste
Estado.

[...]

Art. 76. Quando da passagem das mercadorias ou bens pela primeira repartição fiscal do Estado,
a documentação fiscal correspondente será processada eletronicamente e emitido DARE para recolhimento
do imposto, com vencimento no último dia da segunda quinzena subseqüente à da entrada neste Estado.

[...]


§ 3º. O prazo estabelecido no caput deste artigo não será aplicado aos contribuintes que se
encontrem:


III- com desempenho negativo.

[...]


§ 4º. Para efeito do disposto no inciso III do parágrafo anterior, considera-se desempenho
negativo:

I – a empresa apresentar no ano em curso, ou no anterior, volume de entradas superior ao de
saídas;


In [31]:
def agrupar_notas_mensal(doc, col_base='CNPJ_REM'):
    """ Função para agrupar por mês, por coluna especificada (col_base)"""

    new_doc = doc.groupby([col_base, pd.Grouper(key='DT_EMISSAO', freq='1M')], as_index=False)[['VALOR_BRUTO_ITEM',
                                                                                            'VALOR_DESCONTO_ITEM', 
                                                                                            'VALOR_LIQUIDO_ITEM']].sum()
    
    new_doc.rename(columns={
                            'VALOR_BRUTO_ITEM': 'VALOR_BRUTO',
                            'VALOR_DESCONTO_ITEM': 'VALOR_DESCONTO', 
                            'VALOR_LIQUIDO_ITEM': 'VALOR_LIQUIDO'}, inplace=True
                            )
    
    return new_doc



In [32]:
relatos = {}

# Agrupar dados em base mensal
nfe_entrada_agpt = agrupar_notas_mensal(nfe_entrada, col_base = 'CNPJ_DEST')
nfe_saida_agpt = agrupar_notas_mensal(nfe_saida)
nfce_agpt = agrupar_notas_mensal(nfce)

# Calcular desempenho mensal
saida_agpt = nfe_saida_agpt.merge(nfce_agpt, on = ['CNPJ_REM', 'DT_EMISSAO'], how='outer', suffixes=['', '_NFCE'], validate='one_to_one')
total_agpt = saida_agpt.merge(nfe_entrada_agpt, left_on=['CNPJ_REM', 'DT_EMISSAO'], right_on=['CNPJ_DEST', 'DT_EMISSAO'], how='outer', suffixes=['_NFE_SAIDA', '_NFE_ENTRADA'], validate='one_to_one')

# Alterar nomes de tabelas e manter apenas necessárias
colunas = ['VALOR_BRUTO', 'VALOR_LIQUIDO']
tipos = ['_NFCE', '_NFE_SAIDA', '_NFE_ENTRADA']

colunas_a_manter = ['CNPJ_REM', 'DT_EMISSAO']
colunas_a_manter = colunas_a_manter + [x+y for x in colunas for y in tipos]
total_agpt = total_agpt[colunas_a_manter]

total_agpt.rename(columns={'CNPJ_REM':'CNPJ'}, inplace=True)

# Completando nulos com 0 
total_agpt.fillna(0, inplace=True)

# Calcular desempenho mensal


for col in colunas:
    total_agpt[col+'_RESULTADO'] = total_agpt[col+'_NFCE'] + total_agpt[col+'_NFE_SAIDA'] - total_agpt[col+'_NFE_ENTRADA'] 

total_agpt.sort_values(by='DT_EMISSAO', inplace=True)


# Verificar se há desempenho negativo na maior parte do período e relatar
if len(total_agpt[total_agpt['VALOR_LIQUIDO_RESULTADO']<0]) > (data_fim.year - data_inicio.year)/2 and total_agpt['VALOR_LIQUIDO_RESULTADO'].sum()< -valor_minimo_significancia: 
    msg = f"""
O contribuinte possui {len(total_agpt[total_agpt['VALOR_LIQUIDO_RESULTADO']<0])} períodos com desempenho negativo mensal, somando o equivalente a R$ {total_agpt['VALOR_LIQUIDO_RESULTADO'].sum()} em desempenho no período de {data_inicio.strftime('%d/%m/%Y')} e {data_fim.strftime('%d/%m/%Y')}, conforme anexo {indice_anexo}.
Em virtude disso, e considerando o Art. 75 e 76, §3º e §4º, I, do Decreto 4.335-E/2001, do RICMS-RR (Dec), considera-se que o contribuinte não poderá fazer jus a prazo para pagamento de Antecipação Parcial.
"""
    anexos[f'Anexo {indice_anexo} - Desempenho Mensalizado de Operações com Mercadorias'] = [total_agpt]
    indice_anexo += 1 
    del colunas, colunas_a_manter, tipos, saida_agpt, nfce_agpt, nfe_saida_agpt, nfe_entrada_agpt

else:
    msg = f'Não foi identificada irregularidade com o desempenho operacional do período.'
    del total_agpt, colunas, colunas_a_manter, tipos, saida_agpt, nfce_agpt, nfe_saida_agpt, nfe_entrada_agpt

relatos['Verificação de Desempenho Negativo Mensal por meio das Notas Fiscais de Entrada e Notas Fiscais de Saída ou Ao Consumidor'] = [msg]
print(msg)




explicacao_adicional = f""" Foi acessado o banco de dados da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de obtenção dos documentos fiscais em posse do Fisco. 
Baseado nestes documentos, realizou-se as correções devidas (a fim de que as notas fiscais refletissem o Tipo de Nota Fiscal - Entrada ou Saída - e a Finalidade de Emissão - Normal ou Devolução), bem como a exclusão de documentos com eventos referentes a cancelamento (como, por exemplo, eventos de Não Realização da Operação ou de Desconhecimento). 
Utilizou-se o somatório de documentos fiscais de saída (apenas NF-e e NFC-e) e o somatório de documentos fiscais de entrada, por mês e considerou-se a normal ocorrência de prejuízos em períodos alternados. 
Para fins de classificação de uma situação como irregular, ponderou-se pela presença de desempenhos negativos em pelo menos {(data_fim.year - data_inicio.year)/2} ocorrências. """

explicacoes_adicionais.append(explicacao_adicional)


O contribuinte possui 20 períodos com desempenho negativo mensal, somando o equivalente a R$ -861226.51 em desempenho no período de 01/01/2022 e 31/12/2023, conforme anexo 2.
Em virtude disso, e considerando o Art. 75 e 76, §3º e §4º, I, do Decreto 4.335-E/2001, do RICMS-RR (Dec), considera-se que o contribuinte não poderá fazer jus a prazo para pagamento de Antecipação Parcial.



### 6.2. Verificação de Desempenho Negativo por Produto

In [33]:

# Montar Livro de Entrada e Saída de Produtos baseado em GTIN
entradas = nfe_entrada[['CNPJ_DEST', 'DT_EMISSAO', 'GTIN_ITEM', 'NCM_ITEM', 'DESCRICAO_PROD', 'QTDE_ITEM', 'UND_ITEM', 'VALOR_LIQUIDO_ITEM']]
entradas.rename(columns={'CNPJ_DEST':'CNPJ'}, inplace=True)
entradas['REGIME_COMPETENCIA'] = 'DESPENDIO'
entradas['VALOR_LIQUIDO_ITEM'] = -1 * entradas['VALOR_LIQUIDO_ITEM'] 

saidas = pd.concat([nfe_saida[['CNPJ_REM', 'DT_EMISSAO', 'GTIN_ITEM', 'NCM_ITEM', 'DESCRICAO_PROD', 'QTDE_ITEM', 'UND_ITEM', 'VALOR_LIQUIDO_ITEM']], nfce[['CNPJ_REM', 'DT_EMISSAO', 'GTIN_ITEM', 'NCM_ITEM', 'DESCRICAO_PROD', 'QTDE_ITEM', 'UND_ITEM', 'VALOR_LIQUIDO_ITEM']]])
saidas.rename(columns={'CNPJ_REM':'CNPJ'}, inplace=True)
saidas['REGIME_COMPETENCIA'] = 'RECEITA'

estoque = pd.concat([entradas, saidas])
estoque.sort_values(by='DT_EMISSAO', inplace=True)

# Considerar SAÍDAS como valor POSITIVO e ENTRADAS como valor NEGATIVO (SAÍDA = RECEITA e ENTRADA = DESPÊNDIO)

# Separar por estoque industrializado 
estoque_industrializado = estoque[~estoque['GTIN_ITEM'].isin(['SEM GTIN', 'None'])]
estoque_primario = estoque[estoque['GTIN_ITEM'].isin(['SEM GTIN', 'None'])]
del estoque

In [34]:
# estoque_industrializado['DESEMPENHO_DIARIO_PRODUTO'] = estoque_industrializado.groupby('GTIN_ITEM')['VALOR_LIQUIDO_ITEM'].cumsum()
# estoque_industrializado.sort_values(by=['GTIN_ITEM', 'DT_EMISSAO'], inplace=True)
# # estoque_industrializado.set_index(['CNPJ', 'GTIN_ITEM', 'DT_EMISSAO'], inplace=True)
# estoque_industrializado.to_excel('aaa.xlsx')

In [35]:
# import pandas as pd
# import numpy as np
# import re
# from sklearn.cluster import AgglomerativeClustering
# from sentence_transformers import SentenceTransformer

# # Assuming 'df' is your DataFrame

# # Step 1: Data Preprocessing
# def preprocess_text(text):
#     # Convert to lowercase
#     text = str(text).lower()
#     # Remove special characters and numbers
#     text = re.sub(r'[^a-zA-Z\s]', '', text)
#     # Remove extra spaces
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# def ProcessarDescricoesProdutos(df):
#     df['DESCRICAO_PROD_PROCESSADO'] = df['DESCRICAO_PROD'].apply(preprocess_text)

#     # Step 2: Generate Text Embeddings
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     embeddings = model.encode(df['DESCRICAO_PROD_PROCESSADO'].tolist())

#     # Step 3: Clustering Similar Product Descriptions
#     # Adjust the distance_threshold based on your data
#     clustering_model = AgglomerativeClustering(
#         n_clusters=None,
#         distance_threshold=0.5,
#         metric='cosine',
#         linkage='average'
#     )
#     clustering_model.fit(embeddings)
#     df['Description_Cluster'] = clustering_model.labels_


#     df['ID_PRODUTO_POR_DESCRICAO'] = ['Cluster' + str(line['Description_Cluster']) for _, line in df.iterrows()]
           

#     return df


In [36]:
# estoque_industrializado2 = ProcessarDescricoesProdutos(estoque_industrializado)

# estoque_industrializado2

### Relatar

In [37]:
nome_metodo = 'Auditoria de Operações e Prestações de Saída'
texto = f"No dia {dt.today().strftime('%d/%m/%Y')}, foi realizada auditoria de Operações e Prestações de Saída, por meio das seguintes verificações: \n "

for relato in relatos.keys():
    texto += f'{relato} \n '

existe_explicacoes = False 
texto_adicional = ''
for relato in explicacoes_adicionais:
    texto_adicional += f'{relato} \n'
    existe_explicacoes = True 
if not existe_explicacoes:
    texto_adicional = None

resultado = ''
for relato in relatos.values():
    for resultado_parcial in relato:
        resultado += f'- {resultado_parcial} \n'

del relatos

# Adicionar ao relatório 
doc.inserir_procedimento(nome_metodo=nome_metodo, texto=texto, texto_adicional=texto_adicional, resultado=resultado)

## 7. Auditoria de Faturamento

Objetivos:
- Avaliar se o faturamento declarado é compatível com o somatório de notas fiscais emitidas 
- Avaliar se o faturamento declarado é compatível com a entrada de mercadoria para revenda no período
- Avaliar se o faturamento declarado é compatível com o valor de notas fiscais para uso próprio (uso e consumo e ativo imobilizado)

---
<center> Fundamentos Legais </center>

---

> Resolução CGSN 140/2018

Art. 81. A exclusão do Simples Nacional, mediante comunicação da ME ou da EPP à RFB, em aplicativo disponibilizado no Portal do Simples Nacional, dar-se-á:

[...]

II - obrigatoriamente, quando:

a) a receita bruta acumulada ultrapassar um dos limites previstos no § 1º do art. 2º, hipótese em que a exclusão deverá ser comunicada:

1. até o último dia útil do mês subsequente à ultrapassagem em mais de 20% (vinte por cento) de um desses limites, produzindo efeitos a partir do mês subsequente ao do excesso; ou (Lei Complementar nº 123, de 2006, art. 30, inciso IV, § 1º, inciso IV; art. 31, inciso V, alínea “a”)
2. até o último dia útil do mês de janeiro do ano-calendário subsequente, na hipótese de não ter ultrapassado em mais de 20% (vinte por cento) um desses limites, produzindo efeitos a partir do ano-calendário subsequente ao do excesso; (Lei Complementar nº 123, de 2006, art. 30, inciso IV, § 1º, inciso IV; art. 31, inciso V, alínea “b”)

b) a receita bruta acumulada, no ano-calendário de início de atividade, ultrapassar um dos limites previstos no caput do art. 3º, hipótese em que a exclusão deverá ser comunicada:

1. até o último dia útil do mês subsequente à ultrapassagem em mais de 20% (vinte por cento) de um desses limites, produzindo efeitos retroativamente ao início de atividades; ou (Lei Complementar nº 123, de 2006, art. 30, inciso III, § 1º, inciso III, alínea “a”; art. 31, inciso III, alínea “a”)
2. até o último dia útil do mês de janeiro do ano-calendário subsequente, na hipótese de não ter ultrapassado em mais de 20% (vinte por cento) um desses limites, produzindo efeitos a partir de 1º de janeiro do ano-calendário subsequente; (Lei Complementar nº 123, de 2006, art. 30, inciso III, § 1º, inciso III, alínea “b”; art. 31, inciso III, alínea “b”)

[...]

IV - a partir do próprio mês em que incorridas, hipótese em que a empresa ficará impedida de fazer nova opção pelo Simples Nacional nos 3 (três) anos-calendário subsequentes, nas seguintes hipóteses: (Lei Complementar nº 123, de 2006, art. 29, incisos II a XII e § 1º)

[...]

h) se for constatado que durante o ano-calendário o valor das despesas pagas supera em 20% (vinte por cento) o valor de ingressos de recursos no mesmo período, excluído o ano de início de atividade;

i) se for constatado que durante o ano-calendário o valor das aquisições de mercadorias para comercialização ou industrialização, ressalvadas hipóteses justificadas de aumento de estoque, foi superior a 80% (oitenta por cento) dos ingressos de recursos no mesmo período, excluído o ano de início de atividade;


In [38]:
relatos = {}
explicacoes_adicionais = []

### 7.1. Verificação de Faturamento em relação às Notas Fiscais de saída

#### 7.1.1. Calcular Faturamento via Documentos Fiscais

In [39]:
relatos = {}

# Obter dado total de nfe de saída
nfe_saida_total = nfe_saida[['CHAVE_ACESSO', 'CNPJ_REM', 'DT_EMISSAO', 'VALOR_FINAL_NOTA', 'TIPO_OPER', 'FINALIDADE']].drop_duplicates()

# Excluir devoluções (não compõem faturamento)
nfe_saida_total = nfe_saida_total[nfe_saida_total['FINALIDADE']=='NORMAL']

# Reduzir colunas para somar com nfces
nfe_saida_total = nfe_saida_total[['CHAVE_ACESSO', 'CNPJ_REM', 'DT_EMISSAO', 'VALOR_FINAL_NOTA']]

# Inserir modelo 
nfe_saida_total['MODELO'] = 'NF-e'
nfce['MODELO'] = 'NFC-e'

# Gerar dataframe com saídas
saidas_total = pd.concat([nfe_saida_total, nfce[['CHAVE_ACESSO', 'CNPJ_REM', 'DT_EMISSAO', 'VALOR_FINAL_NOTA', 'MODELO']].drop_duplicates()])
saidas_total.rename(columns={'CNPJ_REM':'CNPJ'}, inplace=True)

# Calcular valor total de faturamento (por meio de documentação de saída)
faturamento_nf = saidas_total.groupby(['CNPJ', pd.Grouper(key='DT_EMISSAO', freq='1M')], as_index=False)[['VALOR_FINAL_NOTA']].sum()
faturamento_nf['DT_EMISSAO'] = faturamento_nf['DT_EMISSAO'].dt.strftime('%m/%Y')

# Alterar nomes
faturamento_nf.columns = ['CNPJ', 'MES_REF', 'VALOR_FATURAMENTO']

faturamento_nf

,CNPJ,MES_REF,VALOR_FATURAMENTO
0,29563548000121,01/2022,64480.83
1,29563548000121,02/2022,60556.72
2,29563548000121,03/2022,54989.60
3,29563548000121,04/2022,55941.09
4,29563548000121,05/2022,67239.14
5,29563548000121,06/2022,69893.87
6,29563548000121,07/2022,73182.41
7,29563548000121,08/2022,96789.70
8,29563548000121,09/2022,87687.44
9,29563548000121,10/2022,104204.40


#### 7.1.2. Identificar faturamento declarado em PGDAS

In [40]:
pgdas_declarados = pgdas.retrieve()

faturamento_pgdas = pgdas_declarados.groupby(['CNPJ', 'dt_ini'])[['Vltotal']].sum()
faturamento_pgdas['dt_ini'] = faturamento_pgdas['dt_ini'].dt.strftime('%m/%Y')

# Alterar nomes de coluna
faturamento_pgdas.columns = ['CNPJ', 'MES_REF', 'VALOR_FATURAMENTO']

NameError: name 'pgdas' is not defined

#### 7.1.3. Cruzar faturamento calculado com faturamento declarado

In [ ]:
cruzamento_faturamento = faturamento_nf.merge(faturamento_pgdas, on=['CNPJ', 'MES_REF'], how='outer', validate='one_to_one', suffixes=['_NF', '_PGDAS'])

cruzamento_faturamento['DIFERENCA_FATURAMENTO'] = cruzamento_faturamento['VALOR_FATURAMENTO_NF'] - cruzamento_faturamento['VALOR_FATURAMENTO_PGDAS']

# Calcular possível valor de irregularidade 
diferenca_fat = round(abs(cruzamento_faturamento[cruzamento_faturamento['DIFERENCA_FATURAMENTO'] >= 0]['DIFERENCA_FATURAMENTO'].sum()), 2)


if (cruzamento_faturamento[cruzamento_faturamento['DIFERENCA_FATURAMENTO'] >= 0].size >= (data_fim.year - data_inicio.year)/2) \
    and diferenca_fat >= valor_minimo_significancia:
    msg = f"Foi identificada irregularidade significativa na comparação entre faturamento declarado e faturamento calculado no montante total de R$ {diferenca_fat} no período fiscalizado, conforme Anexo {indice_anexo}."
    

else:
    msg = f"Não foi identificada irregularidade significativa na comparação entre faturamento declarado e faturamento calculado no período fiscalizado, conforme Anexo {indice_anexo}."

relatos['Verificação de Faturamento declarado em PGDAS em relação às Notas Fiscais de saída'] = [msg]
anexos[f"Anexo {indice_anexo} - Diferença entre faturamentos mensais calculados e declarados"] = [cruzamento_faturamento]
indice_anexo += 1 

del faturamento_nf, faturamento_pgdas

explicacao_adicional = f"""Para a 'Verificação de Faturamento declarado em PGDAS em relação às Notas Fiscais de saída, foi acessado o banco de dados da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de obtenção dos documentos fiscais em posse do Fisco. Para os documentos declaratórios de faturamento mensal, utilizou-se o Portal do Simples Nacional.
Baseado nestes documentos, realizou-se as correções devidas (a fim de que as notas fiscais refletissem o Tipo de Nota Fiscal - Entrada ou Saída - e a Finalidade de Emissão - Normal ou Devolução), bem como a exclusão de documentos com eventos referentes a cancelamento (como, por exemplo, eventos de Não Realização da Operação ou de Desconhecimento). 
Utilizou-se o somatório de documentos fiscais de saída (apenas NF-e e NFC-e), por mês e considerou-se a normal ocorrência de inconsistências entre um mês e outro. 
Para fins de classificação de uma situação como irregular, no que tange ao faturamento declarado e calculado, ponderou-se pela presença de inconsistências em pelo menos {(data_fim.year - data_inicio.year)/2} ocorrências, com somatório atual de irregularidades acima do valor de significância (R$ {valor_minimo_significancia})."""

explicacoes_adicionais.append(explicacao_adicional)

### 7.2. Verificação de Excesso de Aquisições de Mercadorias em Relação ao Faturamento

In [ ]:
# Calcular total de entrada de mercadoria
nfe_entrada_total = nfe_entrada[['CHAVE_ACESSO', 'CNPJ_DEST', 'DT_EMISSAO', 'FINALIDADE', 'CFOP_ITEM', 'VALOR_LIQUIDO_ITEM']]

# Trocar nomes para tornar mais claro
nfe_entrada_total.columns = ['CHAVE_ACESSO', 'CNPJ', 'DT_EMISSAO', 'FINALIDADE', 'CFOP_ITEM', 'VALOR_LIQUIDO_ITEM']

# Excluir devoluções (entradas em virtude de devoluções não deveriam compor o conceito de "aquisições de mercadoria")
nfe_entrada_total = nfe_entrada_total[nfe_entrada_total['FINALIDADE']!='DEVOLUÇÃO']

# Excluir CFOP referente a uso e consumo
CFOP_USO_CONSUMO = [principal+acessorio for principal in ['1', '2', '3', '5', '6', '7'] for acessorio in ['407', '550', '551', '552', '553', '554', '555', '556', '557']]
nfe_entrada_total = nfe_entrada_total[~nfe_entrada_total['CFOP_ITEM'].isin(CFOP_USO_CONSUMO)]

# Obter valor total de aquisições por ano-exercício
nfe_entrada_total = nfe_entrada_total.groupby(['CNPJ', pd.Grouper(key='DT_EMISSAO', freq='1Y')], as_index=False)[['VALOR_LIQUIDO_ITEM']].sum()
nfe_entrada_total.columns = ['CNPJ', 'ANO_REF', 'VALOR_AQUISIÇÕES']

# Alterar formato de data para ANO
nfe_entrada_total['ANO_REF'] = nfe_entrada_total['ANO_REF'].dt.strftime('%Y')

# Obter Faturamento declarado
pgdas_declarados = pgdas.retrieve()

faturamento_pgdas = pgdas_declarados.groupby(['CNPJ', 'dt_ini'])[['Vltotal']].sum()

# Alterar nomes de coluna
faturamento_pgdas.columns = ['CNPJ', 'ANO_REF', 'VALOR_FATURAMENTO']

# Alterar formato de data para ANO
faturamento_pgdas['ANO_REF'] = faturamento_pgdas['ANO_REF'].dt.strftime('%Y')

In [ ]:
# Cruzar faturamento com aquisições
cruzamento_aquisicoes = faturamento_pgdas.merge(nfe_entrada_total, on = ['CNPJ', 'ANO_REF'], how='outer', validate='one_to_one')

cruzamento_aquisicoes['PROPORCAO_AQUISICAO'] = cruzamento_aquisicoes['VALOR_AQUISIÇÕES'] / cruzamento_aquisicoes['VALOR_FATURAMENTO']

# Verificar eventual irregularidade 
if cruzamento_aquisicoes[cruzamento_aquisicoes['PROPORCAO_AQUISICAO']>= 0.8].size > 0:
    msg = """Foi identificado que o contribuinte adquiriu valores superiores a 80% ao faturamento do período no(s) período(s) de: 
        """
    for idx, row in cruzamento_aquisicoes[cruzamento_aquisicoes['PROPORCAO_AQUISICAO']>= 0.8].iterrows():
        msg += '- '
        msg += row['ANO_REF'] 
        msg += f": - R$ {round(row['VALOR_AQUISIÇÕES'], 2)} de aquisições e R$ {round(row['VALOR_FATURAMENTO'], 2)} de faturamento declarado - Proporção de {round(row['PROPORCAO_AQUISICAO'], 2)}"  
        msg += """
        """
    msg += f"Conforme anexo {indice_anexo}"
    
else:
    msg = "Não foi identificado irregularidades na proporção entre as aquisições de mercadoria no período e o faturamento declarado."

relatos['Verificação de Excesso de Aquisições de Mercadorias por Notas Fiscais Eletrônicas em Relação ao Faturamento declarado em PGDAS'] = [msg]
anexos[f"Anexo {indice_anexo} - Proporção de Aquisições em Relação ao Faturamento"] = [cruzamento_aquisicoes]
indice_anexo += 1 



explicacao_adicional = f"""Para a Verificação de Excesso de Aquisições de Mercadorias por Notas Fiscais Eletrônicas em Relação ao Faturamento declarado em PGDAS, foi acessado o banco de dados da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de obtenção dos documentos fiscais em posse do Fisco. Para os documentos declaratórios de faturamento mensal, utilizou-se o Portal do Simples Nacional.
Baseado nestes documentos, realizou-se as correções devidas (a fim de que as notas fiscais refletissem o Tipo de Nota Fiscal - Entrada ou Saída - e a Finalidade de Emissão - Normal ou Devolução), bem como a exclusão de documentos com eventos referentes a cancelamento (como, por exemplo, eventos de Não Realização da Operação ou de Desconhecimento). 
Utilizou-se o somatório de documentos fiscais de aquisições (apenas NF-e) no período (por ano), desde que NÃO possuíssem CFOP de uso e consumo (ou uso como ativo imobilizado). 
Para fins de classificação de uma situação como irregular, para as aquisições de mercadorias em relação ao faturamento, ponderou-se pela presença de inconsistências em pelo menos 1 período de 12 meses, coincidente com o ano-calendário. """

explicacoes_adicionais.append(explicacao_adicional)

### 7.3. Verificação de Excesso de Despesas em Relação ao Faturamento

In [ ]:
# Calcular total de entrada de mercadoria
nfe_entrada_total = nfe_entrada[['CHAVE_ACESSO', 'CNPJ_DEST', 'DT_EMISSAO', 'FINALIDADE', 'CFOP_ITEM', 'VALOR_LIQUIDO_ITEM']]

# Trocar nomes para tornar mais claro
nfe_entrada_total.columns = ['CHAVE_ACESSO', 'CNPJ', 'DT_EMISSAO', 'FINALIDADE', 'CFOP_ITEM', 'VALOR_LIQUIDO_ITEM']

# Excluir devoluções (entradas em virtude de devoluções não deveriam compor o conceito de "aquisições de mercadoria")
nfe_entrada_total = nfe_entrada_total[nfe_entrada_total['FINALIDADE']!='DEVOLUÇÃO']

# Filtrar por CFOP referente a uso e consumo
CFOP_USO_CONSUMO = [principal+acessorio for principal in ['1', '2', '3', '5', '6', '7'] for acessorio in ['407', '550', '551', '552', '553', '554', '555', '556', '557']]
nfe_entrada_total = nfe_entrada_total[nfe_entrada_total['CFOP_ITEM'].isin(CFOP_USO_CONSUMO)]

# Obter valor total de aquisições por ano-exercício
nfe_entrada_total = nfe_entrada_total.groupby(['CNPJ', pd.Grouper(key='DT_EMISSAO', freq='1Y')], as_index=False)[['VALOR_LIQUIDO_ITEM']].sum()
nfe_entrada_total.columns = ['CNPJ', 'ANO_REF', 'VALOR_DESPESAS_USO_CONSUMO']

# Alterar formato de data para ANO
nfe_entrada_total['ANO_REF'] = nfe_entrada_total['ANO_REF'].dt.strftime('%Y')

# Obter Faturamento declarado
pgdas_declarados = pgdas.retrieve()

faturamento_pgdas = pgdas_declarados.groupby(['CNPJ', 'dt_ini'])[['Vltotal']].sum()

# Alterar nomes de coluna
faturamento_pgdas.columns = ['CNPJ', 'ANO_REF', 'VALOR_FATURAMENTO']

# Alterar formato de data para ANO
faturamento_pgdas['ANO_REF'] = faturamento_pgdas['ANO_REF'].dt.strftime('%Y')

In [ ]:
# Cruzar faturamento com aquisições
cruzamento_despesas = faturamento_pgdas.merge(nfe_entrada_total, on = ['CNPJ', 'ANO_REF'], how='outer', validate='one_to_one')

cruzamento_despesas['PROPORCAO_DESPESAS'] = cruzamento_despesas['VALOR_DESPESAS_USO_CONSUMO'] / cruzamento_despesas['VALOR_FATURAMENTO']

# Verificar eventual irregularidade 
if cruzamento_despesas[cruzamento_despesas['PROPORCAO_DESPESAS']>= 0.2].size > 0:
    msg = """Foi identificado que o contribuinte realizou despesas com Uso e Consumo em valores superiores a 20% ao faturamento do período no(s) período(s) de: 
        """
    for idx, row in cruzamento_despesas[cruzamento_despesas['PROPORCAO_DESPESAS']>= 0.2].iterrows():
        msg += '- '
        msg += row['ANO_REF'] 
        msg += f": - R$ {round(row['VALOR_DESPESAS_USO_CONSUMO'], 2)} de despesas e R$ {round(row['VALOR_FATURAMENTO'], 2)} de faturamento declarado - Proporção de {round(row['PROPORCAO_DESPESAS'], 2)}"  
        msg += """
        """
    msg += f"Conforme anexo {indice_anexo}"
    
else:
    msg = "Não foi identificado irregularidades na proporção entre as despesas por produtos de uso e consumo no período e o faturamento declarado."

relatos['Verificação de Excesso de Despesas por meio de Notas Fiscais em Relação ao Faturamento declarado em PGDAS'] = [msg]
anexos[f"Anexo {indice_anexo} - Proporção de Despesas em Relação ao Faturamento"] = [cruzamento_despesas]
indice_anexo += 1 


explicacao_adicional = f"""Para a Verificação de Excesso de Despesas por meio de Notas Fiscais em Relação ao Faturamento declarado em PGDAS, foi acessado o banco de dados da Receita Estadual (SEFAZ-RR) no dia {dt.today().strftime("%d/%m/%Y")} com a finalidade de obtenção dos documentos fiscais em posse do Fisco. Para os documentos declaratórios de faturamento mensal, utilizou-se o Portal do Simples Nacional.
Baseado nestes documentos, realizou-se as correções devidas (a fim de que as notas fiscais refletissem o Tipo de Nota Fiscal - Entrada ou Saída - e a Finalidade de Emissão - Normal ou Devolução), bem como a exclusão de documentos com eventos referentes a cancelamento (como, por exemplo, eventos de Não Realização da Operação ou de Desconhecimento). 
Utilizou-se o somatório de documentos fiscais de aquisições (apenas NF-e) no período (por ano), desde que possuíssem CFOP de uso e consumo (ou ativo imobilizado). 
Para fins de classificação de uma situação como irregular, para as aquisições de mercadorias em relação ao faturamento, ponderou-se pela presença de inconsistências em pelo menos 1 período de 12 meses, coincidente com o ano-calendário. """

explicacoes_adicionais.append(explicacao_adicional)

### Relatar

In [ ]:
nome_metodo = 'Auditoria de Faturamento'
texto = f"No dia {dt.today().strftime('%d/%m/%Y')}, foi realizada auditoria de Faturamento, por meio das seguintes verificações: \n "

for relato in relatos.keys():
    texto += f'{relato} \n '

existe_explicacoes = False 
texto_adicional = ''
for relato in explicacoes_adicionais:
    texto_adicional += f'{relato} \n'
    existe_explicacoes = True 
if not existe_explicacoes:
    texto_adicional = None

resultado = ' '
for relato in relatos.values():
    for resultado_parcial in relato:
        resultado += f'- {resultado_parcial} \n'

del relatos

# Adicionar ao relatório 
doc.inserir_procedimento(nome_metodo=nome_metodo, texto=texto, texto_adicional=texto_adicional, resultado=resultado)

## 8. Conclusão de Relatório

In [41]:
# Concluir relatório
doc.concluir_auditoria()

# Salvar
doc.salvar()